In [1]:
#Book recmmendation system using k nearest neighbor
#dataset from http://www2.informatik.uni-freiburg.de/~cziegler/BX/
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
books=pd.read_csv("E:/project/books/book.csv")
users=pd.read_csv("E:/project/books/user.csv")
ratings=pd.read_csv("E:/project/books/rating.csv")

In [ ]:
#users with less than 200 rating and books with less than 100 rating are excluded
counts1=ratings["userID"].value_counts()
ratings=ratings[ratings["userID"].isin(counts1[counts1>200].index)]
counts=ratings["bookRating"].value_counts()
ratings=ratings[ratings["bookRating"].isin(counts1[counts>=100].index)]

In [ ]:
#merging ratings and book
combine=pd.merge(ratings,book,on='ISBN')
columns=['yearofPublication','publisher','bookAuthor','imageUr15','imageUrIM','imageUrlL']
combine=combine.drop(columns,axis=1)
combine.head()


In [ ]:
#adding new column total rating count
combine=combine.dropna(axis=0,subset=['bookTitle'])
book_ratingCount=(combine_book_rating.groupby(by=['bookTitle'])['bookRating'].count().reset_index().rename(columns={'bookRating':'totalRatingCount'})[['bookTitle','totalRatingCount']])

book_RatingCount.head()

In [ ]:
#combining
rating_with_totalRatingCount=combine.merge(bookRatingCount,left_one='bookTitle',how='left')
rating_with_totalRatingCount.head()

In [ ]:
popularity_threshold=50
rating_popular_book=rating_with_totalRatingCount.query('totalRatingCount>=@popularity_threshold')
rating_popular_book.head()


In [ ]:
#filters to users in us and canada only
combined=rating_popular_book.merge(users,left_on='userID',right_on='userID',how='left')
us_canada_user_rating=combined[combined['location'].str.contains("Usa|Canada")]
us_canada_user_rating=us_canada_user_rating.drop('Age',axis=1)
us_canada_user_rating.head()

In [ ]:
from scipy.sparse import csr_matrix
us_canada_user_rating=us_canada_user_rating.drop_duplicates(['userID','bookTitle'])
us_canada_user_rating_pivot=us_canada_user_rating.pivot(index='bookTitle',columns='userID',values='bookRating').fillna()

us_canada_user_rating_matrix=csr_matrix(us_canada_user_rating_pivot.values)
from sklearn.neighbors import NearestNeighbors

In [ ]:
model=NearestNeighbors(metric='cosine',algorithm='brute')
model.fit(us_canada_user_rating_matrix)